# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
### Author: Fangyan Xu

**The In this project, I use the tools learned about in the lesson to identify lane lines on the road. My pipeline applied to image and video stream is shown as following.**

---

## PipeLine

My pipeline consists of 6 steps. 
<figure>
 <img src="test_images/outputsolidYellowCurve2.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Original image </p> 
 </figcaption>
Step 1, convert the images converted to grayscale 
<figure>
 <img src="report_image/gray.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Grayscale image </p> 
 </figcaption>
Step 2, apply gaussian smoothing on grayscaled images 
Step 3, apply canny edge detection, obtain a set of edges
<figure>
 <img src="report_image/edges.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Edges afte edge detection </p> 
 </figcaption>
Step 4, apply region mask, select edges in desired region
<figure>
 <img src="report_image/masked_edges.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Edges after region mask </p> 
 </figcaption>
Step 5, use Hough transform to find and draw the lines
<figure>
 <img src="test_images/output_segment_solidYellowCurve2.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;">  Line Segment for lane line in original image </p> 
 </figcaption>
</figure>
 <p></p> 
Step 6, add the lines on original image
<figure>
 <img src="report_image/lines_edges.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> full extent of line </p> 
 </figcaption>

# Reflection

### Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import sys
import math
import os

### Helper Functions

Below are some helper functions in my pipeline

In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def Dis(p1, p2):
    return np.sqrt( (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def draw(y1, y2, slop, dis, img, color, thickness):
    xx1 = (y1 - dis) / slop
    yy1 = y1
    xx2 = (y2 - dis) / slop
    yy2 = y2
    cv2.line(img, (int(xx1), int(yy1)), (int(xx2), int(yy2)), color, thickness)

### Revise draw_lines() function

In [3]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    """
    This function is used to draw the full extent of the lane, here we take all edges after
    canny edge detection and compute slop of each edge, group them 
    """
    width = img.shape[1]
    height = img.shape[0]

    upper_limit = 0.61 * height
    lower_limit = height

    x_left_down = 0.052 * width
    x_left_up = 0.448 * width
    x_right_up = 0.5625 * width
    x_right_down = 0.9375 * width

    slops = {} # key : (slop, dis), value: ((x1, y1), (x2, y2))
    slop_threshold = 0.006
    for line in lines:
        for x1,y1,x2,y2 in line:
            # for lines with simlar slop and intercept, only choose the pair of points with longest distance 
            slop = (y1 - y2) / (x1 - x2)
            dis = y1 - slop * x1
            
            insert = 0
            old_keys = slops.keys()
            for s in old_keys:
                if(abs(slop - s[0]) < slop_threshold or abs(dis - s[1]) < 10):
                        insert = 1
                        p1 = slops[s][0][0]
                        p2 = slops[s][0][1]
                        newP1 = ()
                        newP2 = ()
                        
                        # find a pair of points with longest distance
                        x_axis =[]
                        x_axis.append(x1)
                        x_axis.append(x2)
                        x_axis.append(p1[0])
                        x_axis.append(p2[0])
                        
                        points = []
                        points.append((x1, y1))
                        points.append((x2, y2))
                        points.append(p1)
                        points.append(p2)
                        
                        newP1_index = x_axis.index(min(x_axis))
                        newP2_index = x_axis.index(max(x_axis))
                        
                        # record the pair of points and current slop and intercept for computing average later
                        newP1 = points[newP1_index]
                        newP2 = points[newP2_index]

                        newSlop = slops[s][1][0][0] + slop
                        newDis = slops[s][1][0][1] + dis

                        num = slops[s][1][1] + 1
                        del slops[s]
                        slops[(slop, dis)] = ((newP1, newP2), ((newSlop, newDis), num))
                        insert = 1
                        break
            # insert the slop and dis if there is no similar previous one
            if(insert == 0) :
                slops[(slop, dis)] = (((x1, y1), (x2, y2)), ((slop, dis), 1))
                
    right_len = 0
    left_len = 0
    num1 = 0
    num2 = 0

    valid_slops_right = []
    valid_slops_left = []
    valid_dis_right = []
    valid_dis_left = []
    for s in slops.keys():
        
        p1 = slops[s][0][0]
        p2 = slops[s][0][1]
        slop_dis = slops[s][1][0]
        total = slops[s][1][1]
        # compute average slop
        if(p1[0] - p2[0] == 0):
            continue

        slop = slop_dis[0] / total 
        dis = slop_dis[1] / total

        # since the slops of two lane lines have opposite signs, we only need to decide 1 slop for each lane line
        # choose the slop of a line with longest length as the slop of the corresponding lane line
        # since the position of the lane line is relatively fixed, we can limit the value of slop to remove some noise
        leng = Dis(p1, p2)
        
        if((slop > 0.5 and slop < 0.8) or (slop > -0.8 and slop < -0.5)):
            # draw(upper_limit, lower_limit, slop, dis, img, color, thickness) 
            if(slop > 0  and leng > right_len):
                valid_slops_right.append(slop)
                valid_dis_right.append(dis)
            if(slop < 0 and leng > left_len):
                valid_slops_left.append(slop)
                valid_dis_left.append(dis)

    slop_right = 0
    dis_right = 0
    slop_left = 0
    dis_left = 0
    if(len(valid_slops_right) > 0):
        slop_right = np.sum(valid_slops_right) / len(valid_slops_right)
        dis_right = np.sum(valid_dis_right) / len(valid_dis_right)
    if(len(valid_slops_left) > 0):
        slop_left = np.sum(valid_slops_left) / len(valid_slops_left)
        dis_left = np.sum(valid_dis_left) / len(valid_dis_left)
        
    # draw the lines
    if(slop_right != 0):
        draw(upper_limit, lower_limit, slop_right, dis_right, img, color, thickness)
    if(slop_left != 0):
        draw(upper_limit, lower_limit, slop_left, dis_left, img, color, thickness)  
    cv2.imshow('llinees', img)
    # cv2.waitKey(0)

### Find line in an image
This function takes an image as input, find and draw the lines on the input image.

In [4]:
def find_line_in_image(image):
    # global parameter
    width = image.shape[1]
    height = image.shape[0]
    kernel_size = 3
    low_threshold = 50
    high_threshold = 150
    
    # parameters for region mask
    upper_limit = 0.61 * height
    lower_limit = height

    x_left_down = 0.052 * width
    x_left_up = 0.448 * width
    x_right_up = 0.5625 * width
    x_right_down = 0.9375 * width

    cv2.imshow('origin', image)
	#printing out some stats and plotting
    print('This image is:', type(image), 'with dimensions:', image.shape)
    gray = grayscale(image)
    # cv2.imshow('gray', gray)
    
    blur = gaussian_blur(gray, kernel_size)
    # cv2.imshow('blur', blur)
    edges = canny(blur, low_threshold, high_threshold)
    # cv2.imshow('edges', edges)
    
    vertices = np.array([[(x_left_down,lower_limit),(x_left_up, upper_limit), (x_right_up, upper_limit), (x_right_down,lower_limit)]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    # each lane line has two sides, therefore 2 lines will be detected alone each lane line.
    
    rho = 1 
    theta = 1 * np.pi/180 
    threshold = 3    
    min_line_length = 15
    max_line_gap = 10 
    
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    # Draw the lines on the edge image
    lines_edges = weighted_img(line_image, image, 0.8, 1., 0.)
    cv2.imshow('line_edges', lines_edges)
    # cv2.waitKey(3000)
    return lines_edges

### Function for a single image

In [5]:
path = "test_images/"
def single_image(name):
    file = os.path.join(path, name)
    image = cv2.imread(file)
    result = find_line_in_image(image)
    cv2.imshow('result', result)
    plt.imshow(image)
    plt.imshow(result)
    # output_file = os.path.join(path, "output_segment_" + name)
    # cv2.imwrite(output_file,result)
#     cv2.waitKey(0)

### Function for all test images

In [6]:
def for_lines():
    files = os.listdir(path)
    
    for file in files:
        if(file == '.DS_Store' or file == "."):
            continue
        file = os.path.join(path, file)
        image = cv2.imread(file)
        find_line_in_image(image)

### Function for video
Import dependent packages

In [7]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [8]:
video_path = "test_videos/"
def for_video(name):
    input_video = name
    output_video = "output" + name
    white_output = os.path.join(video_path, output_video)
    
    white_input = os.path.join(video_path, input_video)
    clip1 = VideoFileClip(white_input)
    white_clip = clip1.fl_image(find_line_in_image) 
    white_clip.write_videofile(white_output, audio=False)

### Run
Use the function above to run the pipeline on image or video

In [9]:
def main():
    ## test on images 
    single_image("solidYellowCurve2.jpg")
    # single_image("solidWhiteRight.jpg")
    # for_lines()
    
    ## test on video
    for_video("solidWhiteRight.mp4")
    # for_video("solidYellowLeft.mp4")
    
    # for_video("challenge.mp4")

if __name__ == "__main__":
    main()

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
[MoviePy] >>>> Building video test_videos/outputsolidWhiteRight.mp4
[MoviePy] Writing video test_videos/outputsolidWhiteRight.mp4


  1%|▏         | 3/222 [00:00<00:08, 25.69it/s]

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)


  5%|▍         | 10/222 [00:00<00:07, 28.76it/s]

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)


...

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)


 98%|█████████▊| 218/222 [00:08<00:00, 27.54it/s]

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)


100%|█████████▉| 221/222 [00:08<00:00, 28.01it/s]


This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/outputsolidWhiteRight.mp4 



## Shortcomings with your current pipeline

<figure>
 <img src="report_image/corner.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Original image" </p> 
 </figcaption>
 
 <figure>
 <img src="report_image/gray_corner.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Grayscale image </p> 
 </figcaption>

## Possible improvement